# Загрузка датасета и первый взгляд

In [1]:
!mkdir data
!wget https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/hometask/sem4/zhestov.ea%40phystech.edu.csv -O data/zhestov.ea@phystech.edu.csv

--2021-03-04 15:46:09--  https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/hometask/sem4/zhestov.ea%40phystech.edu.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6132 (6.0K) [text/plain]
Saving to: ‘data/zhestov.ea@phystech.edu.csv’

data/zhestov.ea@phy 100%[===================>]   5.99K  --.-KB/s    in 0s      

2021-03-04 15:46:10 (50.1 MB/s) - ‘data/zhestov.ea@phystech.edu.csv’ saved [6132/6132]



In [2]:
import pandas as pd
from scipy import stats as st

In [3]:
data = pd.read_csv('/content/data/zhestov.ea@phystech.edu.csv', header=None)

In [4]:
data.head()

,0,1,2,3,4,5,6,7
0,-1.363188,0.021371,0.320485,0.655761,0.170489,-1.668670,-0.271347,0.517328
1,-0.372176,-0.932204,-0.371692,-0.666623,-1.939313,-0.860681,0.267452,-0.052678
2,-1.154241,0.086681,-0.663427,0.578648,-0.911634,-0.406784,-1.366443,0.114200
3,-1.494851,-2.115424,-0.573977,0.354928,-0.624171,-0.934268,0.188995,0.332669
4,-1.132629,1.256545,-0.228378,1.712289,-2.842673,0.665167,1.232858,0.481192


In [5]:
data.describe()

,0,1,2,3,4,5,6,7
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,-0.302123,-0.497873,-0.667462,0.792811,-0.519058,-0.425253,-0.438858,0.444914
std,1.062679,0.937303,1.104621,0.885732,1.137562,1.059801,1.083716,0.860677
min,-2.146883,-2.115424,-2.720480,-1.140923,-3.021317,-2.712613,-2.351574,-1.983966
25%,-1.059673,-1.340181,-1.124838,0.324511,-1.070533,-1.192344,-0.925668,0.110411
50%,-0.444970,-0.438975,-0.618702,0.704028,-0.545497,-0.481194,-0.476070,0.498146
75%,0.292121,0.087947,-0.187725,1.205233,0.501332,0.533150,0.247838,0.993862
max,2.568914,1.256545,2.127354,2.682470,1.095461,1.602156,1.862021,2.487462


In [21]:
data.shape

(30, 8)

# Задание

## Вопрос 1

Воспользуемся t-критерем о сравнении средних для всех пар из столбцов выборки. Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего?

In [6]:
from itertools import combinations

In [36]:
def pairwise_ttest(data: pd.DataFrame, alpha: float = 0.05) -> int:
    """Returns count of pairwise columns with different means and its pvals."""
    cols = 0
    pvals = []

    for col1, col2 in combinations(data.columns, 2):
        pvalue = st.ttest_ind(data[col1], data[col2])[1]
        pvals.append(pvalue)
        if pvalue <= 0.05:
            cols += 1
    return cols, pvals

In [37]:
print(f'Для {pairwise_ttest(data)[0]} пар')

Для 12 пар.


## Вопрос 2
Значение уровня значимости при использовании поправки Бонферрони ($\alpha=0.05$).

О, я недавно написал Холма-Бонферрони сам на другом курсе...

In [35]:
from statsmodels.stats.multitest import multipletests

In [44]:
pvals = pairwise_ttest(data)[1]

p_new = multipletests(pvals, alpha=0.05, method='bonferroni')[-1]
print(f'Уровень значимости = {p_new:0.3f}')

Уровень значимости = 0.002


## Вопрос 3
Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего при использовании поправки Бонферрони?

In [45]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, alpha=0.05, method='bonferroni')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 11 пар столбцов


## Вопрос 4
Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего при использовании метода Холма?

In [61]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, alpha=0.05, method='holm')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 11 пар столбцов


## Вопрос 5
Оценка $\mu_0$ по методу Шведера-Спьётволла ($\lambda = 0.5$)

In [66]:
def shveder_spjetvoll(p_values: list, lambd_s: float = 0.5) -> float:
    len_bigger_lambd = len(list(filter(lambda x: x > lambd_s, p_values)))
    return (1 / (1 - lambd_s) * (1 + len_bigger_lambd))

In [67]:
pvals = pairwise_ttest(data)[1]

print(f'mu_0 = {shveder_spjetvoll(pvals)}')

mu_0 = 22.0
